In [ ]:
import os 
import cv2
import uuid 
import time
import keras 

In [ ]:
img_path = 'data/images'
number_images = 90

In [ ]:
# cap = cv2.VideoCapture(0)
# for i in range(number_images):
#   ret, frame = cap.read()
#   img_name = os.path.join(img_path, f'{str(uuid.uuid1())}.jpg')
#   cv2.imwrite(img_name, frame)
#   cv2.imshow('frame', frame)
#   time.sleep(0.5)
#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_img(x):
    img = tf.io.read_file(x)
    img =tf.io.decode_jpeg(img)
    return img

In [ ]:
images = tf.data.Dataset.list_files('data/images/*.jpg', shuffle=False)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
img = images.map(load_img)

In [ ]:
plt.imshow(img.as_numpy_iterator().next())

In [ ]:
from sklearn.model_selection import train_test_split
import glob
import shutil

# images_path = glob.glob('data/images/*.jpg')
# labels_path = glob.glob('data/labels/*.json')

# x_train, x_test, y_train, y_test = train_test_split(images_path, labels_path, shuffle=False, train_size=0.85, random_state=0)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, shuffle=False, train_size=0.85, random_state=0)

def split(x, y, c):
    for source_path_img, source_path_label in zip(x, y):
        shutil.copyfile(source_path_img, c + '/' + source_path_img.split('/')[1])
        shutil.copyfile(source_path_label, c + '/' + source_path_label.split('/')[1])

# split(x_train, y_train, 'data/train')
# split(x_test, y_test, 'data/test')
# split(x_val, y_val, 'data/val')

In [ ]:
import albumentations as alb

In [ ]:
cv2.imread('data/images/79400bd4-c452-11ed-abaf-7c214ae279f7.jpg').shape

In [ ]:
aug = alb.Compose([alb.RandomCrop(450, 450),
                  alb.HorizontalFlip(0.5), 
                  alb.RandomBrightnessContrast(0.2), 
                  alb.RandomGamma(0.2), 
                  alb.RGBShift(p=0.5), 
                  alb.VerticalFlip(p=0.5)], 
                bbox_params=alb.BboxParams(format='albumentations', label_fields=['class_labels']))

In [ ]:
img = cv2.imread('data/images/7e4d6fc0-c452-11ed-ad16-7c214ae279f7.jpg')
plt.imshow(img)

In [ ]:
with open('data/labels/7e4d6fc0-c452-11ed-ad16-7c214ae279f7.json', 'r') as f:
    label=json.load(f)

In [ ]:
label['shapes'][0]['points']

In [ ]:
coor = list(np.divide([311.025641025641, 177.43589743589743, 413.0769230769231, 301.53846153846155], [640, 480, 640, 480]))

In [ ]:
augmented = aug(image = img, bboxes=[coor], class_labels=['face'])

In [ ]:
augmented['bboxes']

In [ ]:
cv2.rectangle(augmented['image'], tuple(np.multiply(augmented['bboxes'][0][:2], [450, 450]).astype(int)), 
              tuple(np.multiply(augmented['bboxes'][0][2:], [450, 450]).astype(int)), (0, 0, 255), 4)
plt.imshow(augmented['image'])

In [ ]:
import glob
for label, img in zip(glob.glob('data/labels/*.json'), glob.glob('data/images/*.jpg')):
    imag = cv2.imread(img)
    c = []
    if os.path.exists(label):
        with open(label, 'r') as f:
            label_ = json.load(f)
   
        c.append(label_['shapes'][0]['points'][0][0])
        c.append(label_['shapes'][0]['points'][0][1])
        c.append(label_['shapes'][0]['points'][1][0])
        c.append(label_['shapes'][0]['points'][1][1])
        c = list(np.divide(c, [640,480,640,480]))   
    
    try:
        for i in range(60):
        
            augmen = aug(image = imag, bboxes = [c], class_labels=['face'])
            cv2.imwrite('aug_data/' + f'{img.split("/")[1].split(".")[0]}_{i}.jpg', augmen['image'])
        
            j = {} 
            j['image'] = img.split('\\')[1]
        
            if os.path.exists(label):
                if len(augmen['bboxes']) == 0:
                    j['bbox'] = [0, 0, 0, 0]
                    j['class'] = 0
                else:
                    j['bbox'] = augmen['bboxes'][0]
                    j['class'] = 1
            else: 
                j['bbox'] = [0, 0, 0, 0]
                j['class'] = 0 
   
            with open ('aug_data/' + f'{label.split("/")[1].split(".")[0]}_{i}.json', 'w') as f:
                json.dump(j, f)
    except Exception as e:
            print(e)

In [ ]:
images_path = glob.glob('aug_data/images/*.jpg')
labels_path = glob.glob('aug_data/labels/*.json')

x_train, x_test, y_train, y_test = train_test_split(images_path, labels_path, shuffle=False, train_size=0.85, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, shuffle=False, train_size=0.85, random_state=0)

split(x_train, y_train, "aug_data/train")
split(x_test, y_test, 'aug_data/test')
split(x_val, y_val, 'aug_data/val')



In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle=False)
train_images = train_images.map(load_img)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(load_img)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(load_img)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
i = train.as_numpy_iterator().next()
cv2.rectangle(i[0][1],
                  tuple(np.multiply(i[1][1][1][:2], [120,120]).astype(int)),
                  tuple(np.multiply(i[1][1][1][2:], [120,120]).astype(int)), 
                        (255,0,0), 1)
plt.imshow(i[0][1])

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

In [ ]:
def build_model():
    
    input_layer = Input(shape=(120, 120, 3))
    
    vgg = VGG16(include_top=False)(input_layer)
    
    f1 = GlobalMaxPool2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)
    
    f2 = GlobalMaxPool2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)    
    
    facetracker = Model(inputs = input_layer, outputs = [class2, regress2])
    return facetracker

In [ ]:
train.as_numpy_iterator().next()[1]

In [ ]:
facetracker = build_model()
facetracker.summary()

In [ ]:
x, y = train.as_numpy_iterator().next()

In [ ]:
x.shape

In [ ]:
y

In [ ]:
classes, coords = facetracker.predict(x)

In [ ]:
classes, coords

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1/0.75-1)/batches_per_epoch

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

In [ ]:
def localization_loss(y_true, yhat):
    delta_coord = tf.reduce_sum(tf.square(y_true[:, :2]-yhat[:, :2]))
    
    h_true = y_true[:, 3] - y_true[:, 1]
    w_true = y_true[:, 2] - y_true[:, 0] 
    
    h_pred = yhat[:, 3] - yhat[:, 1]
    w_pred = yhat[:, 2] - yhat[:, 0]     

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true - h_pred))
    
    return delta_coord + delta_size

In [ ]:
classloss = keras.losses.BinaryCrossentropy()
regressloss = localization_loss

In [ ]:
localization_loss(y[1], coords).numpy()

In [ ]:
classloss(y[0], classes).numpy()

In [ ]:
regressloss(y[1], coords)

In [ ]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = FaceTracker(facetracker)

In [ ]:
model.compile(opt, classloss, regressloss)

In [ ]:
logdir='logs'

In [ ]:

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=5, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

In [ ]:
test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = facetracker.predict(test_sample[0])
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
facetracker.save('facetracker.h5')

In [ ]:
facetracker = load_model('facetracker.h5')

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()